### Primary Data Cleaning

This file is to clean and process primary data which involves following steps

1. Standardizing country names
2. Clean the data
3. Converting wide format to long format
4. Merging Parts, Passenger Vehicles and Trucks
5. Writing final data to a csv to be merged with secondary data

In [37]:
# import all the modules needed
import pandas as pd
import country_converter as coco

In [38]:
pv_export_df = pd.read_csv("./../../data/raw/primary/New_PV_Countries_Export_Value.csv",
                           encoding='utf-16',header=0, sep="\t")
pv_export_df.head(5)

,Partner,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Canada,"17,859,000,199","12,652,204,453","16,995,719,535","17,902,530,002","18,776,536,663","20,348,261,635","21,865,353,045","21,023,836,427","22,206,354,040","23,537,578,145","22,743,027,693","23,932,510,142","18,595,463,497","23,706,809,603","26,845,856,308","25,301,461,937","23,242,066,773"
1,Germany,"7,271,930,736","4,359,799,043","3,633,810,812","5,146,003,729","5,857,684,381","4,745,975,309","5,272,829,307","5,910,717,531","6,356,555,766","5,721,504,301","5,416,613,521","6,651,557,562","6,498,904,046","7,258,458,199","7,205,388,022","8,496,591,494","7,538,936,258"
2,Mexico,"4,053,831,183","1,987,600,598","2,842,508,188","3,174,225,028","3,607,341,712","3,631,801,923","3,540,262,939","3,084,575,996","3,604,195,477","3,449,264,042","3,254,667,672","3,227,885,847","2,088,545,024","2,874,156,753","3,606,171,154","4,839,937,174","5,273,695,629"
3,China,"681,596,584","754,526,827","2,921,382,708","4,306,360,159","4,959,652,931","7,531,612,003","9,942,175,219","8,264,903,454","8,211,990,469","9,526,202,379","6,238,289,558","6,984,084,476","5,957,317,167","6,278,802,297","5,024,574,204","6,063,085,338","4,775,689,891"
4,United Arab Emirates,"1,799,478,615","550,584,794","1,045,120,624","1,192,574,437","1,818,622,363","2,141,941,944","2,162,718,008","2,122,690,664","1,605,293,038","1,366,056,839","1,244,937,033","1,154,707,351","811,436,264","1,070,647,419","1,687,900,533","2,166,286,395","2,160,723,518"


In [39]:
def standardize_country(countries):
    """
    Converts a list of country names or codes to their standardized short names.

    Parameters
    ----------
    countries : list or str
        A list of country names or codes, or a single country name/code, to be standardized.

    Returns
    -------
    list or str
        The standardized short names of the input countries. 
        If a single country is provided, returns a string.
        If a list is provided, returns a list of strings.

    Notes
    -----
    This function uses the `coco` library to perform the conversion. 
    The output format is the short name of each country or `not found` 
    if country name/code is not recognized.
    """
    try:
        standardized_name = coco.convert(names=countries, to='name_short')
        if standardized_name == 'not found':
            return countries
        return standardized_name
    except Exception as e:
        print(f"Error converting country name {countries}: {e}")
        return countries

In [40]:
def clean_primary_data(df, country_col, value_col, cols_to_keep,
                       start_year=2008, end_year=2024, not_found_label='not found'):
    """
    Clean and preprocess a dataframe by standardizing country names, 
    filtering columns and rows, and melting into long format.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing data with country and year columns.
    country_col : str
        The name of the column in `df` containing country names to be standardized.
    value_col : str
        The name of the value column in the output DataFrame after melting.
    cols_to_keep : list of str
        The list of columns to keep before melting (e.g., standardized country and year columns).
    start_year : int, optional
        The first year to consider when dropping rows with missing data.
    end_year : int, optional
        The last year to consider when dropping rows with missing data.
    not_found_label : str, optional (default="not found")
        The label used to identify unrecognized countries.

    Returns
    -------
    pd.DataFrame
        A cleaned DataFrame with standardized country names, 
        a 'Year' column, and a column for data values.

    Notes
    -----
    Steps performed:
    1. Drop rows with no data between `start_year` and `end_year`.
    2. Standardize country names using `standardized_country`.
    3. Drop rows where the standardized name equals `not_found_label`.
    4. Retain only the specified columns (`cols_to_keep`).
    5. Melt the DataFrame into long format with `Year`, `standardized_country`, and values.
    """

    # Drop rows with no data in the given year range
    year_cols = [str(year) for year in range(start_year, end_year + 1)]
    df = df.dropna(subset=year_cols, how="all")

    # Standardize country names
    df['standardized_country'] = df[country_col].apply(standardize_country)

    # Drop rows where country name could not be standardized
    df = df[df['standardized_country'] != not_found_label]

    # Keep relevant columns
    df = df.loc[:, cols_to_keep]

    # Melt the dataframe to long format
    melted_df = df.melt(id_vars=['standardized_country', 'Category'],
                        var_name='year', value_name=value_col)

    return melted_df


In [41]:
def get_cols_to_keep(start_year, end_year,country_col='standardized_country',
                     category_col='Category'):
    """
    Generate a list of columns to keep for data cleaning.

    Parameters
    ----------
    start_year : int, optional
        The first year to include in the list of columns.
    end_year : int, optional
        The last year to include in the list of columns.
    string_cols : list of str, optional
        A list of string column names to include before the year columns.

    Returns
    -------
    list of str
        A list of column names including 'standardized_country' and years
        from `start_year` to `end_year`.
    """
    year_cols = [str(year) for year in range(start_year, end_year + 1)]
    cols_to_keep = [country_col, category_col] + year_cols
    return cols_to_keep



In [42]:
pv_export_df["Category"] = "Passenger Vehicles"
pv_export_df.head(5)

,Partner,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Category
0,Canada,"17,859,000,199","12,652,204,453","16,995,719,535","17,902,530,002","18,776,536,663","20,348,261,635","21,865,353,045","21,023,836,427","22,206,354,040","23,537,578,145","22,743,027,693","23,932,510,142","18,595,463,497","23,706,809,603","26,845,856,308","25,301,461,937","23,242,066,773",Passenger Vehicles
1,Germany,"7,271,930,736","4,359,799,043","3,633,810,812","5,146,003,729","5,857,684,381","4,745,975,309","5,272,829,307","5,910,717,531","6,356,555,766","5,721,504,301","5,416,613,521","6,651,557,562","6,498,904,046","7,258,458,199","7,205,388,022","8,496,591,494","7,538,936,258",Passenger Vehicles
2,Mexico,"4,053,831,183","1,987,600,598","2,842,508,188","3,174,225,028","3,607,341,712","3,631,801,923","3,540,262,939","3,084,575,996","3,604,195,477","3,449,264,042","3,254,667,672","3,227,885,847","2,088,545,024","2,874,156,753","3,606,171,154","4,839,937,174","5,273,695,629",Passenger Vehicles
3,China,"681,596,584","754,526,827","2,921,382,708","4,306,360,159","4,959,652,931","7,531,612,003","9,942,175,219","8,264,903,454","8,211,990,469","9,526,202,379","6,238,289,558","6,984,084,476","5,957,317,167","6,278,802,297","5,024,574,204","6,063,085,338","4,775,689,891",Passenger Vehicles
4,United Arab Emirates,"1,799,478,615","550,584,794","1,045,120,624","1,192,574,437","1,818,622,363","2,141,941,944","2,162,718,008","2,122,690,664","1,605,293,038","1,366,056,839","1,244,937,033","1,154,707,351","811,436,264","1,070,647,419","1,687,900,533","2,166,286,395","2,160,723,518",Passenger Vehicles


In [43]:
required_cols = get_cols_to_keep(2008, 2024, 'standardized_country', 'Category')

cleaned_pv_export_df = clean_primary_data(
    pv_export_df,
    country_col='Partner',
    value_col='Export_value',
    # category_col='Category',
    cols_to_keep=required_cols
)

cleaned_pv_export_df.head(5)

Netherlands Antilles not found in regex
French S. Antarctic Terr not found in regex


,standardized_country,Category,year,Export_value
0,Canada,Passenger Vehicles,2008,"17,859,000,199"
1,Germany,Passenger Vehicles,2008,"7,271,930,736"
2,Mexico,Passenger Vehicles,2008,"4,053,831,183"
3,China,Passenger Vehicles,2008,"681,596,584"
4,United Arab Emirates,Passenger Vehicles,2008,"1,799,478,615"


## Similarly load all other export datasets into individual dataframes

In [44]:
trucks_export_df = pd.read_csv("./../../data/raw/primary/MediumHeavy_Export_Countries_Value.csv",
                               encoding='utf-16',header=0, sep="\t")
trucks_export_df.head(5)

,Partner,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Canada,"3,804,919,203","2,563,237,026","3,970,350,074","5,013,353,082","5,557,753,481","5,181,614,688","4,998,684,617","4,568,516,286","3,960,625,475","4,926,195,306","6,239,555,595","6,427,563,893","4,498,169,550","5,418,459,380","6,567,662,285","7,626,994,363","7,476,019,988"
1,Mexico,"183,709,107","96,483,975","137,243,219","147,360,365","211,011,092","271,544,253","394,054,683","326,924,602","369,550,136","227,108,247","268,254,721","320,500,812","202,235,506","207,795,149","304,330,318","459,197,717","806,845,166"
2,Australia,"214,093,395","64,854,255","143,821,288","227,811,124","347,639,980","272,875,584","188,736,657","120,182,897","87,600,541","92,942,497","115,485,977","75,428,802","77,403,523","117,370,471","96,648,954","152,683,161","230,509,977"
3,Ukraine,"595,900","278,635","69,052","114,717",0,0,0,"100,243","704,135","455,140","434,764","85,152","211,145","1,409,910","1,272,213","1,757,591","91,807,324"
4,Israel,"13,833,891","17,717,653","21,323,322","20,667,328","11,039,346","10,025,167","36,893,269","30,213,641","19,284,799","23,806,464","31,877,228","47,485,279","59,502,369","16,481,390","17,251,195","19,894,564","91,349,134"


In [45]:
trucks_export_df["Category"] = "Trucks"
cleaned_trucks_export_df = clean_primary_data(
    trucks_export_df,
    country_col='Partner',
    value_col ='Export_value',
    # category_col='Category',
    cols_to_keep=required_cols
)

cleaned_trucks_export_df.head(5)


Netherlands Antilles not found in regex


,standardized_country,Category,year,Export_value
0,Canada,Trucks,2008,"3,804,919,203"
1,Mexico,Trucks,2008,"183,709,107"
2,Australia,Trucks,2008,"214,093,395"
3,Ukraine,Trucks,2008,"595,900"
4,Israel,Trucks,2008,"13,833,891"


In [46]:
parts_export_df = pd.read_csv("./../../data/raw/primary/Parts_Exports_Countries.csv",
                              encoding='utf-16',header=0, sep="\t")
parts_export_df["Category"] = "Parts"
cleaned_parts_export_df = clean_primary_data(
    parts_export_df,
    country_col='Partner',
    value_col='Export_value',
    # category_col='Category',
    cols_to_keep=required_cols
)
cleaned_parts_export_df.head(5)

French S. Antarctic Terr not found in regex
Netherlands Antilles not found in regex


,standardized_country,Category,year,Export_value
0,Mexico,Parts,2008,"13,971,074,278"
1,Canada,Parts,2008,"28,258,965,666"
2,Australia,Parts,2008,"958,541,210"
3,Germany,Parts,2008,"1,706,706,846"
4,China,Parts,2008,"913,973,328"


#### Merge Export dataframes for passenger vehicles, trucks and parts

In [47]:
# Merge the three cleaned dataframes into one
merged_exports_df = pd.concat([
    cleaned_pv_export_df,
    cleaned_trucks_export_df,
    cleaned_parts_export_df
], ignore_index=True)

merged_exports_df.head()

,standardized_country,Category,year,Export_value
0,Canada,Passenger Vehicles,2008,"17,859,000,199"
1,Germany,Passenger Vehicles,2008,"7,271,930,736"
2,Mexico,Passenger Vehicles,2008,"4,053,831,183"
3,China,Passenger Vehicles,2008,"681,596,584"
4,United Arab Emirates,Passenger Vehicles,2008,"1,799,478,615"


In [48]:
# Check all unique categories in the merged dataframe
merged_exports_df['year'].unique()

array(['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023',
       '2024'], dtype=object)

#### Now load csv files for PV, trucks and parts imports.
#### Then merge the cleaned dataframes to get a single dataframe for imports value

In [49]:
# Load csv file for PV imports
pv_import_df = pd.read_csv("./../../data/raw/primary/New_PV_Countries_Import_Value.csv",
                           encoding='utf-16',header=0, sep="\t")

pv_import_df["Category"] = "Passenger Vehicles"

required_cols = get_cols_to_keep(2008, 2024, 'standardized_country', 'Category')

# Clean the PV imports data
cleaned_pv_import_df = clean_primary_data(
    pv_import_df,
    country_col='Partner',
    value_col='Import_value',
    # category_col='Category',
    cols_to_keep=required_cols
)

cleaned_pv_import_df.head(5)


,standardized_country,Category,year,Import_value
0,Mexico,Passenger Vehicles,2008,"19,795,290,562"
1,Japan,Passenger Vehicles,2008,"40,975,127,629"
2,South Korea,Passenger Vehicles,2008,"7,413,089,740"
3,Canada,Passenger Vehicles,2008,"33,575,452,102"
4,Germany,Passenger Vehicles,2008,"18,282,476,127"


In [50]:
# Load csv file for parts imports
parts_import_df = pd.read_csv("./../../data/raw/primary/Parts_Imports_Countries.csv",
                              encoding='utf-16',header=0, sep="\t")
parts_import_df["Category"] = "Parts"

# Clean the parts imports data
cleaned_parts_import_df = clean_primary_data(
    parts_import_df,
    country_col='Partner',
    value_col='Import_value',
    # category_col='Category',
    cols_to_keep=required_cols
)

cleaned_parts_import_df.head(5)

French S. Antarctic Terr not found in regex
Netherlands Antilles not found in regex


,standardized_country,Category,year,Import_value
0,Mexico,Parts,2008,"25,654,723,740"
1,Canada,Parts,2008,"16,126,706,233"
2,China,Parts,2008,"8,877,534,735"
3,Japan,Parts,2008,"13,157,308,988"
4,South Korea,Parts,2008,"3,893,417,871"


In [51]:
# Load csv file for truck imports
trucks_import_df = pd.read_csv("./../../data/raw/primary/MediumHeavy_Import_Countries_Value.csv",
                               encoding='utf-16', header=0, sep="\t")
trucks_import_df["Category"] = "Trucks"

# Clean the trucks imports data
cleaned_trucks_import_df = clean_primary_data(
    trucks_import_df,
    country_col='Partner',
    value_col='Import_value',
    # category_col='Category',
    cols_to_keep=required_cols
)

cleaned_trucks_import_df.head(5)

,standardized_country,Category,year,Import_value
0,Mexico,Trucks,2008,"2,248,425,964"
1,Canada,Trucks,2008,"2,864,371,851"
2,Japan,Trucks,2008,"298,780,311"
3,Germany,Trucks,2008,"187,825,244"
4,Türkiye,Trucks,2008,"1,386,000"


#### Merge Import dataframes for passenger vehicles, trucks and parts

In [52]:
# Merge the three cleaned import dataframes into one
merged_imports_df = pd.concat([
    cleaned_pv_import_df,
    cleaned_trucks_import_df,
    cleaned_parts_import_df
], ignore_index=True)

merged_imports_df.head()

,standardized_country,Category,year,Import_value
0,Mexico,Passenger Vehicles,2008,"19,795,290,562"
1,Japan,Passenger Vehicles,2008,"40,975,127,629"
2,South Korea,Passenger Vehicles,2008,"7,413,089,740"
3,Canada,Passenger Vehicles,2008,"33,575,452,102"
4,Germany,Passenger Vehicles,2008,"18,282,476,127"


In [53]:
# Merge Export and Import dataframes into one, keeping only rows present in both (no NaN values)
primary_df = pd.merge(
    merged_imports_df,
    merged_exports_df,
    on=["standardized_country", "year", "Category"],
    how="inner",
    suffixes=("_import", "_export")
)

primary_df.head()

,standardized_country,Category,year,Import_value,Export_value
0,Mexico,Passenger Vehicles,2008,"19,795,290,562","4,053,831,183"
1,Japan,Passenger Vehicles,2008,"40,975,127,629","388,126,905"
2,South Korea,Passenger Vehicles,2008,"7,413,089,740","228,776,236"
3,Canada,Passenger Vehicles,2008,"33,575,452,102","17,859,000,199"
4,Germany,Passenger Vehicles,2008,"18,282,476,127","7,271,930,736"


In [54]:
primary_df.to_csv("./../../data/processed/cleaned_primary_trade_data.csv", index=False)